# LLM-архиватор. Эксперименты со steering vector

In [ ]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm.notebook import tqdm
import time
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import gc

In [ ]:
def clear_memory(var_names=None):
    if var_names is None:
        var_names = [
            'model','tokenizer','enc','dec','logits','probs','inp','ctx',
            'ids','recovered_ids','all_bits','all_original_ids'
        ]
    for var in var_names:
        if var in globals():
            del globals()[var]
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()

## Общие и вспомогательные функции кодирования и декодирования

In [ ]:
def probs_to_cdf_int(prob, total=1 << 20):
    cdf = np.cumsum(prob, dtype=np.float64)
    cdf_int = np.floor(cdf * total).astype(np.int64)
    cdf_int = np.maximum.accumulate(cdf_int)
    cdf_int = np.concatenate(([0], cdf_int))
    cdf_int[-1] = total
    return cdf_int

In [ ]:
class ArithmeticEncoder:
    def __init__(self, precision=32):
        self.precision = precision
        self.half  = 1 << (precision - 1)
        self.quarter = self.half >> 1
        self.mask  = (1 << precision) - 1
        self.low   = 0
        self.high  = self.mask
        self.pending = 0
        self.out = []

    def update(self, cdf_low, cdf_high, total):
        # subdivide interval
        rng = self.high - self.low + 1
        self.high = self.low + (rng * cdf_high) // total - 1
        self.low  = self.low + (rng * cdf_low)  // total

        # renormalize
        while True:
            # E1: MSB equal
            if self.high < self.half:
                self._emit(0)
            elif self.low >= self.half:
                self._emit(1)
                self.low  -= self.half
                self.high -= self.half
            # E3: underflow
            elif self.low >= self.quarter and self.high < 3 * self.quarter:
                self.pending += 1
                self.low  -= self.quarter
                self.high -= self.quarter
            else:
                break

            # shift out
            self.low   = (self.low   << 1) & self.mask
            self.high  = ((self.high << 1) & self.mask) | 1

    def _emit(self, bit):
        self.out.append(bit)
        for _ in range(self.pending):
            self.out.append(1 - bit)
        self.pending = 0

    def finish(self):
        self.pending += 1
        if self.low < self.quarter:
            self._emit(0)
        else:
            self._emit(1)
        for _ in range(self.precision):
            self.out.append((self.low >> (self.precision - 1)) & 1)
            self.low = (self.low << 1) & self.mask

In [ ]:
class ArithmeticDecoder:
    def __init__(self, bits, precision=32):
        self.bits = bits
        self.precision = precision
        self.half  = 1 << (precision - 1)
        self.quarter = self.half >> 1
        self.mask  = (1 << precision) - 1

        self.low   = 0
        self.high  = self.mask
        self.value = 0
        self.idx   = 0
        for _ in range(precision):
            self.value = ((self.value << 1) & self.mask) | self._read()

    def _read(self):
        if self.idx < len(self.bits):
            b = self.bits[self.idx]
            self.idx += 1
            return b
        return 0

    def decode(self, cdf_int, total):
        # find symbol
        rng = self.high - self.low + 1
        scaled = ((self.value - self.low + 1) * total - 1) // rng
        symbol = np.searchsorted(cdf_int, scaled, side='right') - 1

        # narrow interval
        c_lo, c_hi = cdf_int[symbol], cdf_int[symbol+1]
        self.high = self.low + (rng * c_hi) // total - 1
        self.low  = self.low + (rng * c_lo) // total

        # renormalize
        while True:
            if self.high < self.half:
                pass
            elif self.low >= self.half:
                self.value -= self.half
                self.low   -= self.half
                self.high  -= self.half
            elif self.low >= self.quarter and self.high < 3 * self.quarter:
                self.value -= self.quarter
                self.low   -= self.quarter
                self.high  -= self.quarter
            else:
                break

            self.low   = (self.low   << 1) & self.mask
            self.high  = ((self.high << 1) & self.mask) | 1
            self.value = ((self.value << 1) & self.mask) | self._read()

        return symbol

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def load_model(model_name: str):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    model.eval()
    model.to(device)

    return tokenizer, model

In [ ]:
def get_data(size: int, tokenizer, path: str = 'enwik8') -> list[int]:
    with open(path, 'rb') as f:
        data = f.read(size)
    text = data.decode('latin-1')
    return data

In [ ]:
def text_to_ids(text: str, tokenizer) -> list[int]:
    return tokenizer.encode(text, add_special_tokens=False)

## Кодирование и декодирование со steering vector.

In [ ]:
def encode_chunks_with_steering(
    data: bytes,
    steering_ids: list[int],
    tokenizer,
    model,
    chunk_size: int = 2000,
    total: int = 1 << 30
):
    chunk_data = []
    start_time = time.perf_counter()
    max_pe = model.config.max_position_embeddings
    steering_len = len(steering_ids)

    for start in tqdm(range(0, len(data), chunk_size), desc='Encoding chunks'):
        chunk = data[start:start + chunk_size]
        text  = chunk.decode('latin-1')
        ids   = tokenizer.encode(text, add_special_tokens=False)

        enc = ArithmeticEncoder()
        for i in tqdm(range(1, len(ids)), desc='  Tokens in chunk', leave=False):
            history = ids[max(0, i - (max_pe - steering_len)):i]
            ctx_ids = steering_ids + history
            inp = torch.tensor([ctx_ids], device=device)

            with torch.no_grad():
                logits = model(inp).logits[0, -1]
                probs  = torch.softmax(logits, dim=-1).cpu().numpy()

            cdf = probs_to_cdf_int(probs, total)
            token_id = ids[i]
            enc.update(cdf[token_id], cdf[token_id + 1], total)

        enc.finish()
        chunk_data.append({'ids': ids, 'bits': enc.out})

    encoding_time = time.perf_counter() - start_time
    print(f"Total encoding time: {encoding_time:.2f} seconds")
    return chunk_data

In [ ]:
def decode_chunks_with_steering(
    chunk_data,
    steering_ids: list[int],
    model,
    total: int = 1 << 30
):
    start_time = time.perf_counter()
    all_recovered_ids = []

    max_pe = model.config.max_position_embeddings
    steering_len = len(steering_ids)

    for entry in tqdm(chunk_data, desc='Decoding chunks'):
        ids  = entry['ids']
        bits = entry['bits']
        dec  = ArithmeticDecoder(bits)

        rec = [ids[0]]
        for i in tqdm(range(1, len(ids)), desc='  Tokens in chunk', leave=False):
            history = ids[max(0, i - (max_pe - steering_len)):i]
            ctx_ids = steering_ids + history
            inp = torch.tensor([ctx_ids], device=device)

            with torch.no_grad():
                logits = model(inp).logits[0, -1]
                probs  = torch.softmax(logits, dim=-1).cpu().numpy()

            cdf = probs_to_cdf_int(probs, total)
            rec.append(dec.decode(cdf, total))

        all_recovered_ids.extend(rec)

    decoding_time = time.perf_counter() - start_time
    print(f"Total decoding time: {decoding_time:.2f} seconds")
    return all_recovered_ids

In [ ]:
def compare_sequences(orig, dec):
    for k, (o, d) in enumerate(zip(orig, dec)):
        if o != d:
            print(f"❌  Расхождение на позиции {k}: orig={o}  decoded={d}")
            return
    if len(orig) != len(dec):
        print(f"❌  Длины списков отличаются: orig={len(orig)}  decoded={len(dec)}")
        return
    print("✅  Совпадают полностью!")

In [ ]:
def compression_stats(data: bytes, enc_out: bytes):
    original_bits = len(data) * 8
    compressed_bits = len(enc_out)
    ratio = compressed_bits / original_bits

    print(f"Исходный размер:   {original_bits} бит")
    print(f"Размер после сжатия: {compressed_bits} бит")
    print(f"Коэффициент сжатия: {ratio:.4f}")

In [ ]:
def decode_text_from_ids(ids, tokenizer) -> str:
    return tokenizer.decode(
        ids,
        clean_up_tokenization_spaces=False,
        skip_special_tokens=False
    )

## Фиксированная строка как steering vector

In [ ]:
tokenizer, model = load_model('EleutherAI/pythia-70m')

In [ ]:
data = get_data(50000, tokenizer)

In [ ]:
steering_ids = text_to_ids("This is Wikipedia html", tokenizer)

In [ ]:
chunk_data = encode_chunks_with_steering(
    data, steering_ids, tokenizer, model)

Encoding chunks:   0%|          | 0/25 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/668 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/732 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/572 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/589 [00:00<?, ?it/s]

...

  Tokens in chunk:   0%|          | 0/494 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/495 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/442 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/452 [00:00<?, ?it/s]

Total encoding time: 162.76 seconds


In [ ]:
recovered_ids = decode_chunks_with_steering(
    chunk_data, steering_ids, model)

Decoding chunks:   0%|          | 0/25 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/668 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/732 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/572 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/589 [00:00<?, ?it/s]

...

  Tokens in chunk:   0%|          | 0/494 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/495 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/442 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/452 [00:00<?, ?it/s]

Total decoding time: 146.35 seconds


In [ ]:
orig_ids = sum([entry['ids'] for entry in chunk_data], [])
encoded_output = sum([entry['bits'] for entry in chunk_data], [])

In [ ]:
compare_sequences(orig_ids, recovered_ids)

✅  Совпадают полностью!


In [ ]:
compression_stats(data, encoded_output)

Исходный размер:   400000 бит
Размер после сжатия: 66406 бит
Коэффициент сжатия: 0.1660


In [ ]:
recovered_text = decode_text_from_ids(recovered_ids, tokenizer)

## Статический soft‑prompt

In [ ]:
def create_soft_prompt(n_soft: int, model):
    H = model.config.hidden_size
    soft_prompt = nn.Parameter(torch.randn(n_soft, H, device=device))
    return soft_prompt

In [ ]:
def encode_chunks_with_soft_prompt_and_embeddings(
    data: bytes,
    tokenizer,
    model,
    soft_prompt: torch.nn.Parameter,
    total: int = 1 << 30,
    chunk_size: int = 2000
):
    chunk_data = []
    start_time = time.perf_counter()
    max_pe = model.config.max_position_embeddings
    n_soft = soft_prompt.shape[0]

    sp = soft_prompt.unsqueeze(0)

    for start in tqdm(range(0, len(data), chunk_size), desc='Encoding chunks'):
        chunk = data[start:start + chunk_size]
        text  = chunk.decode('latin-1')
        ids   = tokenizer.encode(text, add_special_tokens=False)

        enc = ArithmeticEncoder()
        for i in tqdm(range(1, len(ids)), desc='  Tokens in chunk', leave=False):
            history = ids[max(0, i - (max_pe - n_soft)):i]
            input_ids = torch.tensor([history], device=device)
            input_embeds = model.get_input_embeddings()(input_ids)
            inputs_embeds = torch.cat([sp, input_embeds], dim=1)
            attention_mask = torch.ones(inputs_embeds.shape[:2], device=device)

            with torch.no_grad():
                out = model(inputs_embeds=inputs_embeds, attention_mask=attention_mask)
            step_logits = out.logits[0, -1]
            probs = torch.softmax(step_logits, dim=-1).cpu().numpy()
            cdf = probs_to_cdf_int(probs, total)
            token_id = ids[i]
            enc.update(cdf[token_id], cdf[token_id + 1], total)

        enc.finish()
        chunk_data.append({'ids': ids, 'bits': enc.out})

    encoding_time = time.perf_counter() - start_time
    print(f"Total encoding time: {encoding_time:.2f} seconds")
    return chunk_data


In [ ]:
def decode_with_soft_prompt(
    chunk_data,
    model,
    soft_prompt: torch.nn.Parameter,
    total: int = 1 << 30
):
    start_time = time.perf_counter()
    all_rec = []
    max_pe = model.config.max_position_embeddings
    n_soft = soft_prompt.shape[0]
    sp = soft_prompt.unsqueeze(0)

    for entry in tqdm(chunk_data, desc="Decoding chunks"):
        ids  = entry['ids']
        bits = entry['bits']
        dec  = ArithmeticDecoder(bits)

        rec = [ids[0].item() if hasattr(ids[0], 'item') else ids[0]]
        for i in tqdm(range(1, len(ids)), desc='  Tokens in chunk', leave=False):
            hist = rec[max(0, i - (max_pe - n_soft)):i]
            input_ids = torch.tensor(hist, dtype=torch.long, device=device).unsqueeze(0)
            embeds = model.get_input_embeddings()(input_ids)
            inp_emb = torch.cat([sp, embeds], dim=1)
            attn_mask = torch.ones(inp_emb.shape[:2], device=device)

            with torch.no_grad():
                out = model(inputs_embeds=inp_emb, attention_mask=attn_mask)
            logits = out.logits[0, -1]
            probs  = torch.softmax(logits, dim=-1).cpu().numpy()
            cdf    = probs_to_cdf_int(probs, total)
            rec.append(dec.decode(cdf, total))

        all_rec.extend(rec)

    decoding_time = time.perf_counter() - start_time
    print(f"Decoding done in {decoding_time:.2f}s")
    return all_rec

In [ ]:
soft_prompt = create_soft_prompt(200, model)

In [ ]:
chunk_data = encode_chunks_with_soft_prompt_and_embeddings(
    data, tokenizer, model, soft_prompt)

Encoding chunks:   0%|          | 0/25 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/668 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/732 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/572 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/589 [00:00<?, ?it/s]

...

  Tokens in chunk:   0%|          | 0/494 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/495 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/442 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/452 [00:00<?, ?it/s]

Total encoding time: 197.45 seconds


In [ ]:
all_recovered_ids = decode_with_soft_prompt(
    chunk_data, model, soft_prompt)

Decoding chunks:   0%|          | 0/25 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/668 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/732 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/572 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/589 [00:00<?, ?it/s]

...

  Tokens in chunk:   0%|          | 0/494 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/495 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/442 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/452 [00:00<?, ?it/s]

Decoding done in 195.32s


In [ ]:
orig_ids = sum([entry['ids'] for entry in chunk_data], [])
encoded_output = sum([entry['bits'] for entry in chunk_data], [])

In [ ]:
compare_sequences(orig_ids, recovered_ids)

✅  Совпадают полностью!


In [ ]:
compression_stats(data, encoded_output)

Исходный размер:   400000 бит
Размер после сжатия: 68138 бит
Коэффициент сжатия: 0.1703


## Обучаемый soft‑prompt

In [ ]:
class TextChunkDataset(Dataset):
    def __init__(self, path, chunk_size, tokenizer, max_bytes=None):
        raw = open(path, 'rb').read()
        if max_bytes:
            raw = raw[:max_bytes]
        self.chunks = []
        for i in range(0, len(raw), chunk_size):
            text = raw[i:i+chunk_size].decode('latin-1', errors='ignore')
            ids  = tokenizer.encode(text, add_special_tokens=False)
            if len(ids) > 1:
                self.chunks.append(ids)

    def __len__(self):
        return len(self.chunks)

    def __getitem__(self, idx):
        return torch.tensor(self.chunks[idx], dtype=torch.long)

In [ ]:
def create_data_loader(
    tokenizer,
    chunk_size: int,
    max_bytes: int
):
    dataset = TextChunkDataset('enwik8', chunk_size, tokenizer, max_bytes=max_bytes)
    loader = DataLoader(dataset, batch_size=1, shuffle=True)
    return dataset, loader

In [ ]:
def train_soft_prompt(
    loader,
    model,
    soft_prompt: torch.nn.Parameter,
    optimizer,
    epochs: int
):
    n_soft = soft_prompt.size(0)

    for epoch in range(1, epochs + 1):
        total_loss = 0.0
        for batch in tqdm(loader, desc=f"Epoch {epoch}"):
            ids = batch[0].to(device)
            L = ids.size(0)
            if L < 2:
                continue

            optimizer.zero_grad()

            embeds = model.get_input_embeddings()(ids.unsqueeze(0))
            sp     = soft_prompt.unsqueeze(0)
            inp_e  = torch.cat([sp, embeds], dim=1)
            attn   = torch.ones(inp_e.shape[:2], device=device)

            labels = torch.full((1, n_soft + L), -100, dtype=torch.long, device=device)
            labels[0, n_soft:] = ids

            out  = model(inputs_embeds=inp_e, attention_mask=attn, labels=labels)
            loss = out.loss
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg = total_loss / len(loader)
        print(f"Epoch {epoch} done — avg loss: {avg:.6f}")

In [ ]:
def encode_with_soft_prompt(
    dataset,
    model,
    soft_prompt: torch.nn.Parameter,
    total: int = 1 << 30
):
    chunk_data = []
    start_time = time.perf_counter()
    max_pe = model.config.max_position_embeddings
    n_soft = soft_prompt.shape[0]

    for ids in tqdm(dataset, desc="Encoding chunks"):
        ids = ids.to(device)
        enc = ArithmeticEncoder()
        sp = soft_prompt.unsqueeze(0)

        for i in tqdm(range(1, ids.size(0)), desc='  Tokens in chunk', leave=False):
            hist = ids[max(0, i - (max_pe - n_soft)):i].unsqueeze(0)
            embeds = model.get_input_embeddings()(hist)
            inp_emb = torch.cat([sp, embeds], dim=1)
            attn_mask = torch.ones(inp_emb.shape[:2], device=device)

            with torch.no_grad():
                out = model(inputs_embeds=inp_emb, attention_mask=attn_mask)
            logits = out.logits[0, -1]
            probs = torch.softmax(logits, dim=-1).cpu().numpy()

            cdf = probs_to_cdf_int(probs, total)
            token_id = ids[i].item()
            enc.update(cdf[token_id], cdf[token_id + 1], total)

        enc.finish()
        chunk_data.append({
            'ids': ids.cpu(),
            'bits': enc.out
        })

    encoding_time = time.perf_counter() - start_time
    print(f"Encoding done in {encoding_time:.2f}s")
    return chunk_data

### 500 эпох обучения

In [ ]:
soft_prompt = create_soft_prompt(200, model)

In [ ]:
dataset, loader = create_data_loader(tokenizer, chunk_size=2000, max_bytes=50000)

In [ ]:
model.train()
optimizer = torch.optim.Adam([soft_prompt], lr=5e-4)

In [ ]:
train_soft_prompt(loader, model, soft_prompt, optimizer, 500)

Epoch 1:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 1 done — avg loss: 3.711635


Epoch 2:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 2 done — avg loss: 3.689035


Epoch 3:   0%|          | 0/25 [00:00<?, ?it/s]

...

Epoch 498 done — avg loss: 2.915981


Epoch 499:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 499 done — avg loss: 2.915011


Epoch 500:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 500 done — avg loss: 2.914692


In [ ]:
model.eval();

In [ ]:
chunk_data = encode_with_soft_prompt(
    dataset, model, soft_prompt)

Encoding chunks:   0%|          | 0/25 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/668 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/732 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/572 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/589 [00:00<?, ?it/s]

...

  Tokens in chunk:   0%|          | 0/494 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/495 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/442 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/452 [00:00<?, ?it/s]

Encoding done in 217.47s


In [ ]:
all_rec = decode_with_soft_prompt(
    chunk_data, model, soft_prompt)

Decoding chunks:   0%|          | 0/25 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/668 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/732 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/572 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/589 [00:00<?, ?it/s]

...

  Tokens in chunk:   0%|          | 0/494 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/495 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/442 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/452 [00:00<?, ?it/s]

Decoding done in 200.10s


In [ ]:
orig_ids = sum((entry['ids'].tolist() for entry in chunk_data), [])
encoded_output = sum([entry['bits'] for entry in chunk_data], [])

In [ ]:
compare_sequences(orig_ids, all_rec)

✅  Совпадают полностью!


In [ ]:
compression_stats(data, encoded_output)

Исходный размер:   400000 бит
Размер после сжатия: 54256 бит
Коэффициент сжатия: 0.1356


### 100 эпох обучения

In [ ]:
dataset, loader = create_data_loader(tokenizer, chunk_size=2000, max_bytes=50000)

In [ ]:
soft_prompt = create_soft_prompt(200, model)

In [ ]:
model.train()
optimizer = torch.optim.Adam([soft_prompt], lr=5e-4)

In [ ]:
train_soft_prompt(loader, model, soft_prompt, optimizer, 100)

Epoch 1:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 1 done — avg loss: 3.684460


Epoch 2:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 2 done — avg loss: 3.669685


Epoch 3:   0%|          | 0/25 [00:00<?, ?it/s]

...

Epoch 98 done — avg loss: 3.372458


Epoch 99:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 99 done — avg loss: 3.371199


Epoch 100:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 100 done — avg loss: 3.370519


In [ ]:
model.eval();

In [ ]:
chunk_data = encode_with_soft_prompt(
    dataset, model, soft_prompt)

Encoding chunks:   0%|          | 0/25 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/668 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/732 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/572 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/589 [00:00<?, ?it/s]

...

  Tokens in chunk:   0%|          | 0/494 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/495 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/442 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/452 [00:00<?, ?it/s]

Encoding done in 194.38s


In [ ]:
all_rec = decode_with_soft_prompt(
    chunk_data, model, soft_prompt)

Decoding chunks:   0%|          | 0/25 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/668 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/732 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/572 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/589 [00:00<?, ?it/s]

...

  Tokens in chunk:   0%|          | 0/494 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/495 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/442 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/452 [00:00<?, ?it/s]

Decoding done in 195.70s


In [ ]:
orig_ids = sum((entry['ids'].tolist() for entry in chunk_data), [])
encoded_output = sum([entry['bits'] for entry in chunk_data], [])

In [ ]:
compare_sequences(orig_ids, all_rec)

✅  Совпадают полностью!


In [ ]:
compression_stats(data, encoded_output)

Исходный размер:   400000 бит
Размер после сжатия: 62589 бит
Коэффициент сжатия: 0.1565
